# Motion Designator
Motion designator are similar to action designator, but unlike action designator, motion designator represent atomic tasks. While designators usually use vague descriptions, Motion Designators operate with specific parameteres.

Since motion designators perform motions on the robot, we need a robot which we can use. Therefore, we will create a BulletWorld as well as a PR2 robot.

In [ ]:
from pycram.bullet_world import BulletWorld, Object

world = BulletWorld()
pr2 = Object("pr2", "robot", "pr2.urdf")

The following publishes the resources to RvizWeb.
(Skip if running locally.)

In [ ]:
from pycram.ros.tf_broadcaster import TFBroadcaster
from pycram.ros.viz_marker_publisher import VizMarkerPublisher

tf_broadcaster = TFBroadcaster()
viz_publisher = VizMarkerPublisher()

## Move
Move is used to let the robot drive to the given target pose. Motion designators are used in the same way as the other designator, first create a description then resolve it to the actual designator and lastly, perform the resolved designator.

Performing motion designator verifies the given parameters and passes them to the respective robot drivers. 

In [ ]:
from pycram.designators.motion_designator import MoveMotion
from pycram.process_module import simulated_robot
from pycram.pose import Pose

with simulated_robot:
    motion_description = MoveMotion(target=Pose([1, 0, 0], [0, 0, 0, 1]))
    
    motion_desig = motion_description.resolve()
    
    motion_desig.perform()

In [ ]:
world.reset_bullet_world()

## MoveTCP
MoveTCP is used to move the tool center point (TCP) of the given arm to the target position specified by the parameter. Like any designator we start by creating a description and then resolving and performing it.

In [ ]:
from pycram.designators.motion_designator import MoveTCPMotion

with simulated_robot:
    motion_description = MoveTCPMotion(target=Pose([0.5, 0.6, 0.6], [0, 0, 0, 1]), arm="left")
    
    motion_description.resolve().perform()

## Looking
Looking motion designator adjusts the robot state such that the cameras point towards the target pose. Although this motion designator takes the target as position and orientation, in reality only the position is used. 

In [ ]:
from pycram.designators.motion_designator import LookingMotion
from pycram.process_module import simulated_robot

with simulated_robot:
    motion_description = LookingMotion(target=Pose([1, 1, 1], [0, 0, 0, 1]))
    
    motion_description.resolve().perform()

## Move Gripper
Move gripper moves the gripper of an arm to one of two states. The states can be ```open``` and ```close```, which open and close the gripper respectivly.

In [ ]:
from pycram.designators.motion_designator import MoveGripperMotion
from pycram.process_module import simulated_robot

with simulated_robot:
    motion_description = MoveGripperMotion(motion="open", gripper="left")
    
    motion_description.resolve().perform()

## Detecting 
This is the motion designator implementation of detecting, if an object with the given object type is in the field of view (FOV) this motion designator will return an object designator describing the object.

Since we need an object that we can detect, we will spawn a milk for this.

In [ ]:
milk = Object("milk", "milk", "milk.stl", pose=Pose([1.5, 0, 1]))

In [ ]:
from pycram.designators.motion_designator import DetectingMotion, LookingMotion
from pycram.process_module import simulated_robot

with simulated_robot:
    LookingMotion(target=Pose([1.5, 0, 1], [0, 0, 0, 1])).resolve().perform()
    
    motion_description = DetectingMotion(object_type="milk")
    
    obj = motion_description.resolve().perform()
    
    print(obj)

## Move Arm Joints
This motion designator moves one or both arms. Movement targets can either be a dictionary with joint name as key and target pose as value or a pre-defined configuration like 'park'.

In [ ]:
from pycram.designators.motion_designator import MoveArmJointsMotion
from pycram.process_module import simulated_robot

with simulated_robot:
    motion_description = MoveArmJointsMotion(left_arm_config="park", right_arm_poses={"r_shoulder_pan_joint": -0.7})
    
    motion_description.resolve().perform()

## World State Detecting
World state detecting is also used to detect objects, however, the object is not required to be in the FOV of the robot. As long as the object is somewhere in the beliefe state (BulletWorld) a resolved object designator will be returned.

Sine we want to detect something we will spawn an object that we can detect. If you already spawned the milk the the previous example you can skip this step.

In [ ]:
milk = Object("milk", "milk", "milk.stl", pose=Pose([-1, 0, 1]))

In [ ]:
from pycram.designators.motion_designator import WorldStateDetectingMotion
from pycram.process_module import simulated_robot

with simulated_robot:
    motion_description = WorldStateDetectingMotion(object_type="milk")
    
    obj = motion_description.resolve().perform()
    
    print(obj)

## Move Joints
Move joints can move any number of joints of the robot, the designator takes two lists as parameter. The first list are the names of all joints that should be moved and the second list are the positions to which the joints should be moved.

In [ ]:
from pycram.designators.motion_designator import MoveJointsMotion
from pycram.process_module import simulated_robot

with simulated_robot:
    motion_description = MoveJointsMotion(names=["torso_lift_joint", "r_shoulder_pan_joint"], positions=[0.2, -1.2])
    
    motion_description.resolve().perform()